In [1]:
%pip install pycocotools #our annotations are in coco json



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# helpful sources for COCO: https://www.v7labs.com/blog/coco-dataset-guide
# https://docs.aws.amazon.com/rekognition/latest/customlabels-dg/md-coco-overview.html
# https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocotools/coco.py
from pycocotools.coco import COCO
import os
annotation_file = '../Annotations/combined_annotations.coco.json'
coco = COCO(annotation_file)
#now I have to make sure I get the ID for each and every image
image_ids = coco.getImgIds()
images = coco.loadImgs(image_ids)

#preping image data for keras:
for img in images: #iterating over each image in list of images from COCO dataset
    image_id = img['id'] # 'id' is the unique image identifier
    file_name = img['file_name'] #image's file name
    
    annotation_ids = coco.getAnnIds(imgIds=image_id) #this gets the annotation ID"s associated with the given image in the for loop
    the_annotations = coco.loadAnns(annotation_ids)  #loadAnns retrieves annotation data as a LIST of dictionaries
        #each dict includes data on the image like bounding box (bbox)
        
    bboxes = [ann['bbox'] for ann in the_annotations]
    
    #print(bboxes)  ok we have a lot of different numbers here
    

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [3]:
#now I just want to check the total number of images we have
import os
image_dir = '../Image_data/DS 4002 Project 3 raw image data'
# get a simple list of all files
all_files = os.listdir(image_dir)
# Filter out non-image files incase i skrewed something up
image_files = [f for f in all_files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

num_images = len(image_files) #this gives the total count of images we have
print(f'Total number of images: {num_images}')
print("great, we have 1601 images in our repo!")



Total number of images: 1601
great, we have 1601 images in our repo!


In [4]:
#ok, now I'll split our data into an 20/80 train/test split using sklearn
import sklearn.model_selection
from sklearn.model_selection import train_test_split
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)
print(f'Number of training images: {len(train_files)}')
print(f'Number of validation images: {len(val_files)}')

Number of training images: 1280
Number of validation images: 321


In [ ]:


#next I'm going to use cv2, which is helpful for doing things like image resizing
#source: https://konfuzio.com/en/cv2/#:~:text=The%20cv2%20module%20is%20the,commonly%20used%20functions%20in%20cv2.
%pip install opencv-python-headless
import cv2
import numpy as np
import matplotlib.pyplot as plt
#I will start by initializing each images' width and height in a list
widths = []
heights = []

for file in image_files:
    image_path = os.path.join(image_dir, file) #make sure paths are consistent across systems, in case this is replicated elsewhere
        #again, this is image_dir: image_dir = '../Image_data/DS 4002 Project 3 raw image data'
    image = cv2.imread(image_path) 
        #print(image) --> did this to check, this is a 3 element list of info abt each image
    h, w = image.shape[:2]  #for each image we get height and width, the :2 is to extract only these first two elements
    widths.append(w)  # Add width to the list
    heights.append(h)  # Add height to the list

widths = np.array(widths)
heights = np.array(heights)

print(f"Testing first 10 widths and heights. \n Widths: {widths[:10]}, \n Heights: {heights[:10]}")
print(f"length of each. Length Widths: {len(widths)}, Length of Heights: {len(heights)}")
#ok sweet looks like everything is working thusfar. We now have the pixel widths and height for every image



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Testing first 10 widths and heights. 
 Widths: [612 262 612 612 800 480 480 600 275 612], 
 Heights: [408 192 408 409 533 270 320 424 183 370]
length of each. Length Widths: 1601, Length of Heights: 1601


In [ ]:
#now i'll make a histogram including the sizes of the images

import os
import matplotlib.pyplot as plt

# Define the path to the outputs folder
output_dir = '../Output'

# Plot histograms of image widths and heights
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(widths, bins=30, color='blue', alpha=0.7)
plt.title('Distribution of Image Widths')
plt.xlabel('Width (pixels)')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
plt.hist(heights, bins=30, color='green', alpha=0.7)
plt.title('Distribution of Image Heights')
plt.xlabel('Height (pixels)')
plt.ylabel('Frequency')

plt.tight_layout()

# Save the histogram to the outputs folder
output_path = os.path.join(output_dir, 'image_sizes_histogram.png')
plt.savefig(output_path)  # Save the plot as a PNG file
plt.close()  # Close the plot to free memory

print(f"Histogram is visibile Output folder")


Histogram saved to ../Output/image_sizes_histogram.png


In [ ]:
#now for some basic statistics on the images
from scipy import stats

mode_width = int(stats.mode(widths)[0])
mode_height = int(stats.mode(heights)[0])
print(f"The Mode dimensions respectively: {mode_width}, {mode_height}")

print("\n")
avg_pxl_width = int(np.mean(widths))
avg_pxl_height = int(np.mean(heights))
print(f"the average dimensions of an image in our dataset is: \n{avg_pxl_width}, {avg_pxl_height}")




The Mode dimensions respectively: 612, 408


the average dimensions of an image in our dataset is: 
616, 454
